In [30]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, BasicTicker
from bokeh.transform import transform
from bokeh.palettes import Blues9
from bokeh.palettes import Iridescent18
from bokeh.io import export_png
import datetime

# 读取CSV文件
# 如果你的CSV文件已经读取为df，可以跳过这一步
# df = pd.read_csv('your_file.csv')

# Hyper parameters
## Define directories
SAVE_DIR = "./pics/"
DATA_DIR = "../../data_sf/"
CSV_NAME = 'combined_crime_data.csv'

# read it once only
df = pd.read_csv(DATA_DIR + CSV_NAME)

# 假设df已经存在，如你所说
# 首先过滤出"motor vehicle theft"类型的犯罪
motor_theft_df = df[df['Incident Category'].str.lower() == 'motor vehicle theft'].copy()

# 确保日期列是日期类型
motor_theft_df['Incident Date'] = pd.to_datetime(motor_theft_df['Incident Date'])

# 创建年份和月份列
motor_theft_df['Year'] = motor_theft_df['Incident Date'].dt.year
motor_theft_df['Month'] = motor_theft_df['Incident Date'].dt.month

# 获取数据中的最小年份和最大年份
min_year = 2003  # 你可以替换为你数据集的实际最小年份
max_year = 2025  # 你可以替换为你数据集的实际最大年份

# 创建各年各月的计数
monthly_counts = motor_theft_df.groupby(['Year', 'Month']).size().reset_index(name='count')

# 创建完整的年月索引（包括没有数据的月份）
years = range(min_year, max_year + 1)
months = range(1, 13)

# 创建所有可能的年月组合
all_year_month = pd.MultiIndex.from_product([years, months], names=['Year', 'Month']).to_frame(index=False)

# 合并实际数据和所有可能的年月，填充缺失值为0
monthly_counts = pd.merge(all_year_month, monthly_counts, on=['Year', 'Month'], how='left').fillna(0)

# 将count列转换为整数类型
monthly_counts['count'] = monthly_counts['count'].astype(int)

# 创建热力图所需的数据格式
# 将月份数字转换为月份名称以便在图中显示
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
monthly_counts['Month_Name'] = monthly_counts['Month'].map(month_names)

# 创建数据源
source = ColumnDataSource(
    data=dict(
        year=[str(year) for year in monthly_counts['Year']],
        month=monthly_counts['Month_Name'],
        month_idx=monthly_counts['Month'],
        count=monthly_counts['count']
    )
)

# 创建颜色映射器 - 从浅蓝到深蓝
# colors = list(reversed(Blues9))  # 反转颜色，使得高值为深色
colors = list(Iridescent18)
mapper = LinearColorMapper(palette=colors, low=0, high=monthly_counts['count'].max())

# 设置图形尺寸和工具
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

# 创建年月标签
years = [str(year) for year in range(min_year, max_year + 1)]
months = list(month_names.values())

# 创建热力图
p = figure(
    # title="Monthly Motor Vehicle Theft (2003-2025)",
    toolbar_location=None,
    y_range=months,
    x_range=list(years),  # 反转顺序，使得最早的年份在底部
    x_axis_location="above",
    width=600,
    height=600,
    tools=TOOLS,
    # toolbar_location='below',
    tooltips=[('Year-Month', '@year-@month'), ('Count', '@count')],
    background_fill_color=None,  # 透明背景
    border_fill_color=None,      # 透明边框
    outline_line_color=None      # 透明轮廓
)

# 添加热力图矩形
p.rect(
    y="month",
    x="year",
    width=1,
    height=1,
    source=source,
    fill_color=transform('count', mapper),
    line_color=None
)

# 添加颜色条
color_bar = ColorBar(
    color_mapper=mapper,
    ticker=BasicTicker(desired_num_ticks=7),
    label_standoff=6,
    border_line_color=None,
    location=(0, 0),
    width=20,
    height=300,                        # 调整为更短的高度
    title="",                          # 移除默认标题
    scale_alpha=1,                  # no稍微透明化颜色条
    background_fill_color=None, 
    # major_label_text_font_style='bold',
    
)

p.add_layout(color_bar, 'right')

# 设置坐标轴
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0
# p.title.text_font_size = '12pt'

# 将x轴（年份）标签设置为倾斜显示
p.xaxis.major_label_orientation = 0.9  # 大约40度角

# 设置网格线为透明
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# p.xaxis.major_label_text_font_style = 'bold'  # 加粗x轴(年份)标签
# p.yaxis.major_label_text_font_style = 'bold'  # 加粗y轴(月份)标签


# 保存为HTML文件（背景透明）
output_file("motor_vehicle_theft_heatmap.html")
save(p)

# 注意：如果你想导出为PNG，你需要安装额外的依赖
# 确保已安装selenium和chromedriver
# export_png(p, filename="motor_vehicle_theft_heatmap.png", webdriver=webdriver)

'/Users/msrtea7/Private/MASTER_COURSE_socialDataAnalysisVisualization/ASS_2/02806_assignment2/visualizations/heatmap/motor_vehicle_theft_heatmap.html'